Simple example of eXirt's use

Run all bullets in the sequence below!



Generate files?


## Install dependences

In [ ]:
!pip install catsim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install openml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.5 MB/s eta 0:00:00
  Created wheel for openml: filename=openml-0.13.1-py3-none-any.whl size=142784 sha256=e4b6c2dc7d2547e1ec62da3079f5e5677b6f1914872d98368aaa953824c74906
  Stored in directory: /root/.cache/pip/wheels/f7/7a/fa/f73edc5ccecbab13bbf5fb888f2f943720f77114a1fa4ceffe
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=b6ae9879309572955060bfcaeda7bb5379d728432734f0b1f90580a71e427a73
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built openml liac-arff


In [ ]:
import openml
import pandas as pd
import statistics
import numpy as np
import random
import copy


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from google.colab import files

## Install eXirt 

In [ ]:
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py

--2023-05-29 19:38:54--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7453 (7.3K) [text/plain]
Saving to: ‘decodIRT_MLtIRT.py’

decodIRT_MLtIRT.py  100%[===================>]   7.28K  --.-KB/s    in 0s      

2023-05-29 19:38:54 (53.3 MB/s) - ‘decodIRT_MLtIRT.py’ saved [7453/7453]

--2023-05-29 19:38:54--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awa

In [ ]:
!pip install eXirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for eXirt: filename=eXirt-1.0.1.4-py3-none-any.whl size=19096 sha256=9869ed1f7b6ba48510ec1faf2bdafe26f65d274f88af36ef43848525f4e7ff37
  Stored in directory: /root/.cache/pip/wheels/fc/fb/40/74d60ddc3f9f8bb04ffa5b471c1716348dfdff314deffd16f0
Successfully built eXirt


In [ ]:
from pyexirt import eXirt as explainer

## Load data and Pre-process

In [ ]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(len(df_norm[column].unique()) > 1): #fix NaN generation
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        else:
          df_norm[column] = 0
    return df_norm

In [ ]:
#Select dataset name by openml link https://www.openml.org/search?sort=date
dataset_name = "qsar-biodeg"

In [ ]:
#load dataset by OpenML

dataset = openml.datasets.get_dataset(dataset_name)
X, Y, categorical_indicator, attribute_names = dataset.get_data(
                  dataset_format="dataframe", target=dataset.default_target_attribute)

print(dataset)
#Preprocess Y and X numerics

if (Y.dtype != 'numeric'):
  Y = Y.astype(int)

for i,c in enumerate(X.columns):
  if (X[c].dtype != 'float64'):
    X = X.astype(float)

#Normalization
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) # 70% training and 30% test



OpenML Dataset
Name..........: qsar-biodeg
Version.......: 1
Format........: ARFF
Upload Date...: 2015-05-25 21:14:53
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/1592286/qsar-biodeg.arff
OpenML URL....: https://www.openml.org/d/1494
# of features.: 42
# of instances: 1055


## Creation and prediction model

In [ ]:
model = RandomForestClassifier(1000)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

## Global explanation rank

In [ ]:
explainer = explainer.eXirt()

In [ ]:
global_explanation_attributes, global_explanation_attributes_scores = explainer.explainRankByEXirt(model, X_train, X_test, y_train, y_test,dataset_name)

In [ ]:
global_explanation_attributes

['V36',
 'V1',
 'V18',
 'V39',
 'V38',
 'V16',
 'V34',
 'V37',
 'V22',
 'V10',
 'V8',
 'V27',
 'V15',
 'V2',
 'V13',
 'V17',
 'V28',
 'V11',
 'V12',
 'V7',
 'V14',
 'V30',
 'V33',
 'V5',
 'V31',
 'V41',
 'V3',
 'V9',
 'V6',
 'V40',
 'V35',
 'V4',
 'V23',
 'V32',
 'V26',
 'V25',
 'V24',
 'V29',
 'V20',
 'V19',
 'V21']

In [ ]:
global_explanation_attributes_scores

,Final Score
V36,-108.979004
V1,-108.902198
V18,-108.750242
V39,-108.628758
V38,-108.583145
V16,-108.552754
V34,-108.522403
V37,-108.492006
V22,-108.431356
V10,-108.416192
